In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (SparkSession.builder.appName("Analyzing the vocabulary of Pride and Prejudice.").getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/03 18:08:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sparkContext.setLogLevel("WARN")

In [10]:
book = spark.read.text("./1342-0.txt")

In [11]:
book

DataFrame[value: string]

In [14]:
print(book.head())

Row(value='The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen')


In [15]:
spark?

Type:        SparkSession
String form: <pyspark.sql.session.SparkSession object at 0x7fb4c47f3130>
File:        ~/environment/hello_pyth/de-lab/delab-env/lib/python3.9/site-packages/pyspark/sql/session.py
Docstring:  
The entry point to programming Spark with the Dataset and DataFrame API.

A SparkSession can be used create :class:`DataFrame`, register :class:`DataFrame` as
tables, execute SQL over tables, cache tables, and read parquet files.
To create a :class:`SparkSession`, use the following builder pattern:

.. autoattribute:: builder
   :annotation:

Examples
--------
>>> spark = SparkSession.builder \
...     .master("local") \
...     .appName("Word Count") \
...     .config("spark.some.config.option", "some-value") \
...     .getOrCreate()

>>> from datetime import datetime
>>> from pyspark.sql import Row
>>> spark = SparkSession(sc)
>>> allTypes = sc.parallelize([Row(i=1, s="string", d=1.0, l=1,
...     b=True, list=[1, 2, 3], dict={"s": 0}, row=Row(a=1),
...     time=datetim

In [16]:
book.show(15, truncate=100)

+--------------------------------------------------------------------+
|                                                               value|
+--------------------------------------------------------------------+
|  The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen|
|                                                                    |
|    This eBook is for the use of anyone anywhere at no cost and with|
|almost no restrictions whatsoever.  You may copy it, give it away or|
| re-use it under the terms of the Project Gutenberg License included|
|                      with this eBook or online at www.gutenberg.org|
|                                                                    |
|                                                                    |
|                                          Title: Pride and Prejudice|
|                                                                    |
|                                                 Author: Jane Austen|
|     